In [1]:
import dash
from dash import html, dcc, Input, Output, State, ALL, dash_table, MATCH
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import base64
import io
from flask_caching import Cache

# -----------------------------
# Global Data & Configuration
# -----------------------------
df = pd.DataFrame()
numeric_cols = []
categorical_cols = []
xy_charts = ["bar", "line", "scatter", "multi_line", "area"]
cat_val_charts = ["pie", "funnel", "heatmap"]

# -----------------------------
# Dash App
# -----------------------------
app = dash.Dash(__name__)
server = app.server

# -----------------------------
# Server-Side Cache Configuration
# -----------------------------
cache = Cache(app.server, config={
    'CACHE_TYPE': 'FileSystemCache',
    'CACHE_DIR': 'cache-directory',
    'CACHE_DEFAULT_TIMEOUT': 300
})

# -----------------------------
# Helper Function for File Upload
# -----------------------------
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    
    try:
        if 'csv' in filename:
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            return html.Div('Invalid file type.')
            
        for col in df.columns:
            if df[col].dtype == 'object' and df[col].nunique() / len(df) < 0.2:
                df[col] = df[col].astype('category')
        
        return df
    except Exception as e:
        print(e)
        return html.Div('There was an error processing this file.')

# -----------------------------
# Layout
# -----------------------------
app.layout = html.Div([
    html.H2("📊 Interactive Dashboard", style={
        "textAlign": "center",
        "margin": "20px 0",
        "font-family": "Calibri, sans-serif",
        "color": "#2C3E50"
    }),

    # -----------------------------
    # File Upload Section
    # -----------------------------
    html.Div([
        dcc.Upload(
            id='upload-data',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select a File')
            ]),
            style={
                'width': '50%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px auto',
                'cursor': 'pointer'
            },
            multiple=False
        ),
        html.Div(id='upload-status', style={'textAlign': 'center', 'margin-top': '10px'}),
    ], style={'textAlign': 'center', 'margin-bottom': '20px'}),

    # -----------------------------
    # Dimension Filters
    # -----------------------------
    html.Div(id='dimension-filters-container', style={"display":"flex", "flex-wrap":"wrap", "justify-content":"center", "margin-bottom":"30px"}),

    # -----------------------------
    # KPI Cards Section
    # -----------------------------
    html.Div([
        html.Button("Add KPI Card", id="add-card-btn", n_clicks=0, style={
            "background-color":"#3498DB", "color":"white", "border":"none", "border-radius":"5px", "padding":"6px 16px", "font-size":"13px", "cursor":"pointer", "margin-bottom":"10px", "font-family": "Calibri, sans-serif"
        })
    ], style={"textAlign": "center"}),
    html.Div(id="card-container", style={"display": "flex", "justify-content": "center", "flex-wrap": "wrap", "gap": "20px", "margin-bottom":"30px"}),

    # -----------------------------
    # Table Charts Section
    # -----------------------------
    html.Div([
        html.Button("Add Table Chart", id="add-table-btn", n_clicks=0, style={
            "background-color":"#1ABC9C", "color":"white", "border":"none", "border-radius":"5px", "padding":"6px 16px", "font-size":"13px", "cursor":"pointer", "margin-bottom":"10px", "font-family": "Calibri, sans-serif"
        }),
        html.Div(id="table-chart-container", style={"display":"flex", "flex-wrap":"wrap", "width":"100%", "gap":"20px"})
    ], style={"margin-bottom":"30px"}),

    # -----------------------------
    # Category-Value Charts Section
    # -----------------------------
    html.Div([
        html.Button("Add Category-Value Chart", id="add-catval-btn", n_clicks=0, style={
            "background-color":"#9B59B6", "color":"white", "border":"none", "border-radius":"5px", "padding":"6px 16px", "font-size":"13px", "cursor":"pointer", "margin-bottom":"10px", "font-family": "Calibri, sans-serif"
        }),
        html.Div(id="catval-chart-container", style={"display":"flex", "flex-wrap":"wrap", "justify-content":"center", "gap":"20px"})
    ], style={"margin-bottom":"30px"}),

    # -----------------------------
    # XY Charts Section
    # -----------------------------
    html.Div([
        html.Button("Add XY Chart", id="add-xy-btn", n_clicks=0, style={
            "background-color":"#E67E22", "color":"white", "border":"none", "border-radius":"5px", "padding":"6px 16px", "font-size":"13px", "cursor":"pointer", "margin-bottom":"10px", "font-family": "Calibri, sans-serif"
        }),
        html.Div(id="xy-chart-container", style={"display":"flex", "flex-wrap":"wrap", "justify-content":"center", "gap":"20px"})
    ]),

    # -----------------------------
    # Data Stores
    # -----------------------------
    dcc.Store(id="filter-state", data={})
], style={"padding":"20px", "background-color":"#F4F6F7", "font-family": "Calibri, sans-serif"})


# -----------------------------
# Callbacks
# -----------------------------
@app.callback(
    Output("upload-status", "children"),
    Output("dimension-filters-container", "children"),
    Input("upload-data", "contents"),
    State("upload-data", "filename")
)
def update_uploaded_data_and_filters(contents, filename):
    if contents:
        df_uploaded = parse_contents(contents, filename)
        if isinstance(df_uploaded, pd.DataFrame):
            cache.set('uploaded-data-key', df_uploaded.to_json())
            
            global numeric_cols, categorical_cols
            numeric_cols = df_uploaded.select_dtypes(include='number').columns.tolist()
            all_categorical_cols = df_uploaded.select_dtypes(include=['object', 'datetime', 'category']).columns.tolist()
            categorical_cols = [col for col in all_categorical_cols if df_uploaded[col].nunique() <= 20]
            
            filter_children = [
                html.Div([
                    html.Label(col, style={"font-weight":"bold", "margin-bottom":"5px", "font-size":"13px", "font-family": "Calibri, sans-serif"}),
                    dcc.Dropdown(
                        id={'type':'filter','col':col},
                        options=[{"label": "Select All", "value": "ALL"}] + [{"label": v, "value": v} for v in sorted([str(x) for x in df_uploaded[col].unique() if pd.notna(x)])],
                        value="ALL",
                        multi=True,
                        style={"width":"180px", "border-radius":"5px", "padding":"2px", "font-size":"12px", "font-family": "Calibri, sans-serif"}
                    )
                ], style={"margin-right":"15px", "margin-bottom":"10px"}) for col in categorical_cols
            ]
            
            return f"File '{filename}' uploaded successfully.", filter_children
        else:
            return df_uploaded, []
    return "Please upload a CSV or Excel file to begin.", []


@app.callback(
    Output({'type': 'filter', 'col': MATCH}, 'value'),
    Input({'type': 'filter', 'col': MATCH}, 'value'),
    State({'type': 'filter', 'col': MATCH}, 'options'),
    prevent_initial_call=True
)
def handle_select_all_ui(selected_values, all_options):
    if not selected_values or "ALL" in selected_values:
        if len(selected_values) > 1:
            selected_values.remove("ALL")
            return selected_values
        elif len(selected_values) == 0:
            return ["ALL"]
    return selected_values


@app.callback(
    Output("filter-state", "data", allow_duplicate=True),
    Input({'type':'filter','col':ALL}, 'value'),
    State("filter-state", "data"),
    prevent_initial_call=True
)
def update_filter_state(values, current_state):
    ctx = dash.callback_context
    if not ctx.triggered:
        return current_state
    
    # Get the ID of the filter that triggered the callback
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
    triggered_col = eval(triggered_id)['col'] # Use eval to convert string ID to dict

    # Find the corresponding value from the list of all input values
    triggered_values = values[
        next(i for i, item in enumerate(ctx.args_grouping[0]) if item['id']['col'] == triggered_col)
    ]

    new_state = current_state.copy()
    
    if triggered_values is None or "ALL" in triggered_values:
        new_state[triggered_col] = None
    else:
        new_state[triggered_col] = triggered_values
    
    return new_state


# -----------------------------
# KPI Cards
# -----------------------------
@app.callback(
    Output("card-container", "children"),
    Input("add-card-btn", "n_clicks"),
    State("card-container", "children"),
    prevent_initial_call=True
)
def update_cards(add_clicks, children):
    if children is None: children = []
    
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json: return children
    
    df_uploaded = pd.read_json(df_uploaded_json)
    global numeric_cols
    numeric_cols = df_uploaded.select_dtypes(include='number').columns.tolist()
    
    if len(children) >= len(numeric_cols): return children
    card_index = len(children)
    default_col = numeric_cols[card_index] if numeric_cols else None
    value = df_uploaded[default_col].sum() if not df_uploaded.empty and default_col else 0

    new_card = html.Div([
        dcc.Dropdown(
            id={'type':'card-measure','index':card_index},
            options=[{"label": c, "value": c} for c in numeric_cols],
            value=default_col,
            clearable=False,
            style={"width":"120px"}
        ),
        dcc.Graph(
            id={'type':'card-graph','index':card_index},
            figure=go.Figure(go.Indicator(mode="number", value=value, title={"text": default_col})),
            style={"height":"120px","width":"150px"}
        )
    ], style={"display":"flex","flex-direction":"column","align-items":"center", "border":"1px solid #666","padding":"10px","border-radius":"5px", "background-color":"#f9f9f9"})
    children.append(new_card)
    return children


@app.callback(
    Output({'type': 'card-graph', 'index': MATCH}, 'figure'),
    Input({'type': 'card-measure', 'index': MATCH}, 'value'),
    State("filter-state", "data")
)
def update_single_card_value(selected_col, filter_state):
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json:
        return go.Figure(go.Indicator(mode="number", value=0, title={"text": "No Data"}))
        
    df_filtered = pd.read_json(df_uploaded_json)
    
    # Apply filters
    for col, val_list in filter_state.items():
        if val_list and col in df_filtered.columns:
            df_filtered = df_filtered[df_filtered[col].astype(str).isin(val_list)]

    if not selected_col:
        return go.Figure(go.Indicator(mode="number", value=0, title={"text": "No Data"}))
        
    val = df_filtered[selected_col].sum() if not df_filtered.empty else 0
    fig = go.Figure(go.Indicator(mode="number", value=val, title={"text": selected_col}))
    fig.update_layout(margin=dict(l=5,r=5,t=5,b=5))
    return fig


# -----------------------------
# Table Chart
# -----------------------------
@app.callback(
    Output("table-chart-container","children"),
    Input("add-table-btn","n_clicks"),
    State("table-chart-container","children"),
    prevent_initial_call=True
)
def add_table_chart(n_clicks, children):
    if children is None: children=[]
    chart_index = len(children)
    new_chart = html.Div([
        html.Label("Aggregated Table", style={"font-weight":"bold", "font-family":"Calibri, sans-serif"}),
        dash_table.DataTable(
            id={'type':'table-chart-datatable','index':chart_index},
            data=[],
            columns=[],
            sort_action="native",
            style_table={"overflowX": "auto", "maxHeight":"400px", "overflowY":"auto"},
            style_cell={"textAlign": "center", "font-family": "Calibri, sans-serif", "padding": "5px", "minWidth": "80px", "width": "80px", "maxWidth": "150px"},
            style_header={"backgroundColor": "#2C3E50", "color": "white", "fontWeight": "bold", "border": "1px solid #ddd", "border-radius":"5px"},
            style_data={"backgroundColor": "white", "color": "#2C3E50", "border": "1px solid #ddd", "border-radius":"5px"}
        )
    ], style={
        "border":"1px solid #666", "padding":"10px", "border-radius":"8px", "background-color":"#f9f9f9", "width":"100%", "margin-bottom":"20px", "box-shadow":"2px 2px 5px rgba(0,0,0,0.1)"
    })
    children.append(new_chart)
    return children


@app.callback(
    Output({'type': 'table-chart-datatable', 'index': ALL}, 'data'),
    Output({'type': 'table-chart-datatable', 'index': ALL}, 'columns'),
    Input("filter-state", "data")
)
def update_table_data(filter_state):
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json:
        return [[]] * len(dash.callback_context.outputs_list[0]), [[]] * len(dash.callback_context.outputs_list[1])

    dff = pd.read_json(df_uploaded_json)
    
    # Apply filters
    for col, val_list in filter_state.items():
        if val_list and col in dff.columns:
            dff = dff[dff[col].astype(str).isin(val_list)]
    
    dimensions = [col for col in dff.select_dtypes(include=['object', 'datetime', 'category']).columns if dff[col].nunique() <= 20]
    measures = dff.select_dtypes(include='number').columns.tolist()

    data_list = []
    columns_list = []

    if not dimensions or not measures:
        columns_out = [{"name": c, "id": c} for c in dff.columns]
        data_list.append(dff.to_dict('records'))
        columns_list.append(columns_out)
    else:
        dff_cleaned = dff.dropna(subset=dimensions + measures)
        if dff_cleaned.empty:
            data_list.append([])
            columns_list.append([])
        else:
            agg_dict = {measure: ['sum', 'mean'] for measure in measures}
            aggregated_df = dff_cleaned.groupby(dimensions).agg(agg_dict).reset_index()
            aggregated_df.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in aggregated_df.columns]
            table_cols = []
            for dim in dimensions:
                table_cols.append({"name": dim, "id": dim})
            for measure in measures:
                table_cols.append({"name": f"{measure} (Sum)", "id": f"{measure}_sum", "type": "numeric"})
                table_cols.append({"name": f"{measure} (Average)", "id": f"{measure}_mean", "type": "numeric", "format": dash_table.FormatTemplate.money(2)})
            data_list.append(aggregated_df.to_dict('records'))
            columns_list.append(table_cols)
    
    return data_list, columns_list


# -----------------------------
# Category-Value Charts
# -----------------------------
@app.callback(
    Output("catval-chart-container","children"),
    Input("add-catval-btn","n_clicks"),
    State("catval-chart-container","children"),
    prevent_initial_call=True
)
def add_catval_chart(n_clicks, children):
    if children is None: children=[]
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json: return children
    
    df_uploaded = pd.read_json(df_uploaded_json)
    numeric_cols = df_uploaded.select_dtypes(include='number').columns.tolist()
    categorical_cols = df_uploaded.select_dtypes(include=['object', 'datetime', 'category']).columns.tolist()
    
    chart_index = len(children)
    new_chart = html.Div([
        html.Label("Category-Value Chart Type"),
        dcc.Dropdown(
            id={'type':'catval-chart-type','index':chart_index},
            options=[{"label": c, "value": c} for c in cat_val_charts],
            value="pie",
            style={"width":"150px"}
        ),
        html.Label("Category"),
        dcc.Dropdown(
            id={'type':'catval-chart-category','index':chart_index},
            options=[{"label": c, "value": c} for c in categorical_cols],
            value=categorical_cols[0] if categorical_cols else None,
            style={"width":"150px"}
        ),
        html.Label("Value"),
        dcc.Dropdown(
            id={'type':'catval-chart-value','index':chart_index},
            options=[{"label": c, "value": c} for c in numeric_cols],
            value=numeric_cols[0] if numeric_cols else None,
            style={"width":"150px"}
        ),
        dcc.Graph(id={'type':'catval-chart-graph','index':chart_index}, style={"height":"250px","width":"400px"})
    ], style={"border":"1px solid #666","padding":"10px","border-radius":"5px","background-color":"#f9f9f9"})
    children.append(new_chart)
    return children


@app.callback(
    Output({'type': 'catval-chart-graph', 'index': MATCH}, 'figure'),
    Input({'type': 'catval-chart-type', 'index': MATCH}, 'value'),
    Input({'type': 'catval-chart-category', 'index': MATCH}, 'value'),
    Input({'type': 'catval-chart-value', 'index': MATCH}, 'value'),
    State("filter-state", "data")
)
def update_single_catval_fig(selected_type, selected_cat, selected_val, filter_state):
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json:
        return go.Figure()

    df_filtered = pd.read_json(df_uploaded_json)

    # Apply filters
    for col, val_list in filter_state.items():
        if val_list and col in df_filtered.columns:
            df_filtered = df_filtered[df_filtered[col].astype(str).isin(val_list)]

    if not selected_cat or not selected_val:
        return go.Figure()
    
    try:
        df_agg = (
            df_filtered.groupby(selected_cat)[selected_val]
            .sum()
            .reset_index()
            .sort_values(selected_val, ascending=False)
        )

        if len(df_agg) > 20:
            df_agg = df_agg.head(20)

        if selected_type == "pie":
            return px.pie(df_agg, names=selected_cat, values=selected_val)
        elif selected_type == "funnel":
            return px.funnel(df_agg, y=selected_cat, x=selected_val)
        elif selected_type == "heatmap":
            return px.density_heatmap(df_filtered, x=selected_cat, y=selected_val)
        else:
            return go.Figure()
    except Exception as e:
        print(f"Error creating chart: {e}")
        return go.Figure()


# -----------------------------
# XY Charts
# -----------------------------
@app.callback(
    Output("xy-chart-container","children"),
    Input("add-xy-btn","n_clicks"),
    State("xy-chart-container","children"),
    prevent_initial_call=True
)
def add_xy_chart(n_clicks, children):
    if children is None: children=[]
    
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json: return children
    
    df_uploaded = pd.read_json(df_uploaded_json)
    numeric_cols = df_uploaded.select_dtypes(include='number').columns.tolist()
    
    chart_index = len(children)
    new_chart = html.Div([
        html.Label("XY Chart Type"),
        dcc.Dropdown(
            id={'type':'xy-chart-type','index':chart_index},
            options=[{"label": c, "value": c} for c in xy_charts],
            value="bar",
            style={"width":"150px"}
        ),
        html.Label("X-axis"),
        dcc.Dropdown(
            id={'type':'xy-chart-x','index':chart_index},
            options=[{"label": c, "value": c} for c in df_uploaded.columns],
            value=df_uploaded.columns[0] if df_uploaded.columns.any() else None,
            style={"width":"150px"}
        ),
        html.Label("Y-axis"),
        dcc.Dropdown(
            id={'type':'xy-chart-y','index':chart_index},
            options=[{"label": c, "value": c} for c in numeric_cols],
            value=numeric_cols[0] if numeric_cols else None,
            style={"width":"150px"}
        ),
        dcc.Graph(id={'type':'xy-chart-graph','index':chart_index}, style={"height":"250px","width":"400px"})
    ], style={"border":"1px solid #666","padding":"10px","border-radius":"5px","background-color":"#f9f9f9"})
    children.append(new_chart)
    return children


@app.callback(
    Output({'type': 'xy-chart-graph', 'index': MATCH}, 'figure'),
    Input({'type': 'xy-chart-type', 'index': MATCH}, 'value'),
    Input({'type': 'xy-chart-x', 'index': MATCH}, 'value'),
    Input({'type': 'xy-chart-y', 'index': MATCH}, 'value'),
    State("filter-state", "data")
)
def update_single_xy_chart(selected_type, selected_x, selected_y, filter_state):
    df_uploaded_json = cache.get('uploaded-data-key')
    if not df_uploaded_json:
        return go.Figure()
        
    df_filtered = pd.read_json(df_uploaded_json)

    # Apply filters
    for col, val_list in filter_state.items():
        if val_list and col in df_filtered.columns:
            df_filtered = df_filtered[df_filtered[col].astype(str).isin(val_list)]

    if not selected_x or not selected_y:
        return go.Figure()
    
    try:
        df_agg = (
            df_filtered.groupby(selected_x)[selected_y]
            .sum()
            .reset_index()
            .sort_values(selected_y, ascending=False)
        )

        if len(df_agg) > 50:
            df_agg = df_agg.head(50)

        if selected_type == "bar":
            return px.bar(df_agg, x=selected_x, y=selected_y)
        elif selected_type in ["line", "multi_line"]:
            return px.line(df_agg, x=selected_x, y=selected_y)
        elif selected_type == "scatter":
            return px.scatter(df_agg, x=selected_x, y=selected_y)
        elif selected_type == "area":
            return px.area(df_agg, x=selected_x, y=selected_y)
        else:
            return go.Figure()
            
    except Exception as e:
        print(f"Error creating chart: {e}")
        return go.Figure()


# -----------------------------
# Run App
# -----------------------------
if __name__ == "__main__":
    app.run(debug=True)